## CMPE-272 Big Data
### Team #5


### Dataset information:
#### Name: Digital Music (reviews_Digital_Music_5.json)
#### Source: http://jmcauley.ucsd.edu/data/amazon/index.html
#### Sample schema:
```
{
  "reviewerID": "A2SUAM1J3GNN3B",
  "asin": "0000013714",
  "reviewerName": "J. McDonald",
  "helpful": [2, 3],
  "reviewText": "I bought this for my husband who plays the piano.  He is having a wonderful time playing these old hymns.  The music  is at times hard to read because we think the book was published for singing from more than playing from.  Great purchase though!",
  "overall": 5.0,
  "summary": "Heavenly Highway Hymns",
  "unixReviewTime": 1252800000,
  "reviewTime": "09 13, 2009"
}
```

In [93]:

import ibmos2spark

# @hidden_cell
credentials = {
    'endpoint': 'https://s3-api.us-geo.objectstorage.service.networklayer.com',
    'api_key': '4_r22NrNCwbJmK_Ldw0SidA6tJ2CBQMHwDb0ufQHXKa2',
    'service_id': 'iam-ServiceId-3999fa4b-f2fc-446e-a22a-b3049f3fde46',
    'iam_service_endpoint': 'https://iam.ng.bluemix.net/oidc/token'}

configuration_name = 'os_3f30c173f8f14d6faa1b70b233b44a49_configs'
cos = ibmos2spark.CloudObjectStorage(sc, credentials, configuration_name, 'bluemix_cos')

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
# Since JSON data can be semi-structured and contain additional metadata, it is possible that you might face issues with the DataFrame layout.
# Please read the documentation of 'SparkSession.read()' to learn more about the possibilities to adjust the data loading.
# PySpark documentation: http://spark.apache.org/docs/2.0.2/api/python/pyspark.sql.html#pyspark.sql.DataFrameReader.json

df_data_1 = spark.read.json(cos.url('reviews_Digital_Music_5.json', 'cmpe272bigdata-donotdelete-pr-ajvhabxitk4408'))
df_data_1 = df_data_1.withColumn("helpful_rating", df_data_1["helpful"].getItem(0)/ df_data_1["helpful"].getItem(1))
df_data_1.take(1)


[Row(asin='5555991584', helpful=[3, 3], overall=5.0, reviewText='It\'s hard to believe "Memory of Trees" came out 11 years ago;it has held up well over the passage of time.It\'s Enya\'s last great album before the New Age/pop of "Amarantine" and "Day without rain." Back in 1995,Enya still had her creative spark,her own voice.I agree with the reviewer who said that this is her saddest album;it is melancholy,bittersweet,from the opening title song."Memory of Trees" is elegaic&majestic.;"Pax Deorum" sounds like it is from a Requiem Mass,it is a dark threnody.Unlike the reviewer who said that this has a "disconcerting" blend of spirituality&sensuality;,I don\'t find it disconcerting at all."Anywhere is" is a hopeful song,looking to possibilities."Hope has a place" is about love,but it is up to the listener to decide if it is romantic,platonic,etc.I\'ve always had a soft spot for this song."On my way home" is a triumphant ending about return.This is truly a masterpiece of New Age music,a mu

### Dataset count:

In [94]:
df_data_1.count()


64706

### Top 10 albums with the best raitings and the most helpful comments:

In [98]:
from pyspark.sql.functions import *

df_data_1.filter("overall = 5.0 and helpful_rating = 1.0") \
         .groupBy("asin") \
         .agg({"helpful_rating": "count"}) \
         .sort("count(helpful_rating)", ascending=False).take(10) \


[Row(asin='B000003AEK', count(helpful_rating)=69),
 Row(asin='B0000025RI', count(helpful_rating)=65),
 Row(asin='B000001A5X', count(helpful_rating)=62),
 Row(asin='B000002NJS', count(helpful_rating)=53),
 Row(asin='B000002OPL', count(helpful_rating)=50),
 Row(asin='B000001Y15', count(helpful_rating)=48),
 Row(asin='B00003002C', count(helpful_rating)=47),
 Row(asin='B00000163G', count(helpful_rating)=46),
 Row(asin='B000023VR6', count(helpful_rating)=45),
 Row(asin='B000002LGQ', count(helpful_rating)=43)]

### What else to query?